# Notes:

Metrics to add:
- column for type of workout so we can predict in different workouts


Time sequence options:
- pad at the end of each sequence and build an LSTM RNN with a layer that ignores padded portions of the sequence and a bidirectional layer to retain context from each phase
    1) separate pitch into phases (wind up, sit down power, and throw maybe? or possibly: wind-up, stride, arm cocking, acceleration, release)
        - ENSURE phases are always started the same, this takes away the noise of different starts and is ESSENTIAL or just do tree based instead of lstm
        - Add phase ID (e.g., wind_up=0, acceleration=1, release=2) as a feature to the model.
    2) use time column to split into different sequences (per trial)
        - consider normalizing the time sequences to 0,1. This only makes sense if it's not already normalized, check first and check distance in time steps to ensure no noise
    3) pad at the end of each phase so that it each sequence matches the max sequence
        - Pad each phase separately to its own max length (e.g., wind-up padded to 20 steps, acceleration to 50 steps).
        - Use a hierarchical model (e.g., separate LSTM per phase) to avoid mixing padded values across phases.
    4) create masked neural network that is bidirectional for edge cases and masking so we exclude the padded steps
        For causal tasks, use unidirectional LSTMs.
        For non-causal tasks (e.g., post-pitch classification), use bidirectional layers.
            from tensorflow.keras.layers import Input, Masking, Bidirectional, LSTM, Dense

            # Phase-specific input (e.g., wind-up)
            inputs = Input(shape=(max_phase_length, num_features + 2))  # +2 for phase ID and normalized time
            masked = Masking(mask_value=-1.0)(inputs)  # Assume padded time = -1
            lstm_out = Bidirectional(LSTM(64))(masked)  
            outputs = Dense(1, activation='sigmoid')(lstm_out)
        Add attention mechanisms to explicitly highlight critical moments (e.g., release):
            python
            from tensorflow.keras.layers import Attention, Concatenate
            # Add attention to focus on key timesteps (e.g., release)
            context_vector = Attention()([lstm_out, lstm_out])
            outputs = Concatenate()([lstm_out, context_vector])


Other options to test
- Dynamic Time Warping (DTW): Align sequences non-linearly to a reference length.
    Use Case: Best for comparing shapes of biomechanical curves (e.g., elbow angle vs. time).
    Use DTW as a preprocessing step for similarity-based tasks (e.g., clustering pitches by motion pattern).

- Resampling: Interpolate sequences to a fixed length (e.g., 100 steps) using biomechanical curves’ shape.
    Use Case: Suitable when biomechanical patterns are time-invariant (e.g., joint angles during release).
    Use cubic spline interpolation instead of linear to preserve curve shape.


Key Tests to Validate Approach

    Phase Alignment Check:

        Plot the distribution of phase durations (e.g., wind-up: 0.2–0.3s, release: 0.1–0.15s).

        Ensure padding doesn’t exceed 20% of the phase’s natural duration.
Key Tests to Validate Approach

    Phase Alignment Check:

        Plot the distribution of phase durations (e.g., wind-up: 0.2–0.3s, release: 0.1–0.15s).

        Ensure padding doesn’t exceed 20% of the phase’s natural duration.

    Ablation Study:

        Compare performance of:
        a) Phase-padded LSTM
        b) Resampled LSTM
        c) DTW-aligned model

    Attention Visualization:

        Use libraries like tf-explain to confirm attention focuses on biomechanically critical moments (e.g., release).

    Noise Injection Test:

        Add Gaussian noise to padded regions and verify model robustness (accuracy shouldn’t degrade)
    Ablation Study:

        Compare performance of:
        a) Phase-padded LSTM
        b) Resampled LSTM
        c) DTW-aligned model

    Attention Visualization:

        Use libraries like tf-explain to confirm attention focuses on biomechanically critical moments (e.g., release).

    Noise Injection Test:

        Add Gaussian noise to padded regions and verify model robustness (accuracy shouldn’t degrade)

# Load in the datasets: summarized, emg left joined, bio interpolated

In [ ]:
import pandas as pd

# Load EMG pitch data for validation not for use really
emg_pitch_data = pd.read_parquet('../data/processed/emg_pitch_data_processed.parquet')
print("\nEMG Pitch Data Columns and Unique Values:")
for col in emg_pitch_data.columns:
    print(f"\n{col}:")
    print(f"Unique values: {emg_pitch_data[col].unique()}")

# Load summarized biomechanical profile
summarized_bio = pd.read_parquet('../data/processed/ml_datasets/summarized/final_summarized_biomechanical_profile.parquet')
print("\nSummarized Biomechanical Profile Columns and Unique Values:") 
for col in summarized_bio.columns:
    print(f"\n{col}:") 
    print(f"Unique values: {summarized_bio[col].unique()}")


# Granular Datasets:
# Resample/interpolate the biomechanics dataset to join evenly onto the EMG data: No changes
# Bio dataset with EMG filtered out dataset: is_interpolated filter: filter for non interpolated data if you want to take away emg data for a bio dataset without interpolated added columns (will filter out EMG so they are on the bio frequency)
# EMG dataset with phases added on: create a interpolated column list so we can differ that from the non and create a EMG dataset with pitch phases added on (creating the simplistic EMG dataset with phases added on, no fake data involved and straight to the muscles dataset)

# Load granular dataset with interpolated bio data
granular_data = pd.read_parquet('../data/processed/ml_datasets/final_inner_join_emg_biomech_data.parquet')
print("\nGranular Dataset Columns and Unique Values:")
for col in granular_data.columns:
    print(f"\n{col}:")
    print(f"Unique values: {granular_data[col].unique()}")








EMG Pitch Data Columns and Unique Values:

EMG 1 (mV) - FDS (81770):
Unique values: [-0.0391089 -0.0345769 -0.0142672 ...  0.0040284  0.1359579  0.0691539]

ACC X (G) - FDS (81770):
Unique values: [-0.9207153 -0.9168091 -0.8994141 ...  0.4370117 -1.8967285 -1.4609985]

ACC Y (G) - FDS (81770):
Unique values: [0.406311  0.4068604 0.4025269 ... 0.1681519 0.414917  0.5172119]

ACC Z (G) - FDS (81770):
Unique values: [-0.3459473 -0.3414917 -0.3353271 ...  0.190918   0.2297974  0.1813354]

GYRO X (deg/s) - FDS (81770):
Unique values: [ 33.5343513  32.5190849  32.        ... -92.5190811  57.7557259
  44.0534363]

GYRO Y (deg/s) - FDS (81770):
Unique values: [  -6.2442746   -7.8091602   -9.2748089 ... -236.870224  -212.427475
 -107.2900772]

GYRO Z (deg/s) - FDS (81770):
Unique values: [-17.1832066 -17.4503822 -17.5496178 ...  69.3587799  64.8473282
  95.5267181]

EMG 1 (mV) - FCU (81728):
Unique values: [-0.008896   0.0159457  0.0287022 ...  0.336202   0.0241703 -0.207126 ]

ACC X (G) - FCU

EDA
Check all the basics:
- p value for each metric for suspicious stuff
- outliers
- standardization
- normalization
- dispersion
- disparity
- nulls and which to take out or to impute
- anything else that fixes this dataset 

In [10]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.decomposition import PCA

def detailed_eda(df, dataset_name="Dataset", display_plots=False):
    """
    Perform a detailed Exploratory Data Analysis (EDA) on the given DataFrame.
    
    Additional features added:
    - Duplicate row checks.
    - Consistency checks for datetime columns.
    - Visual Exploratory Analysis (histograms, density plots, Q-Q plots, boxplots).
    - Multicollinearity analysis using Variance Inflation Factor (VIF).
    - Basic time series checks: Stationarity (ADF test) and ACF/PACF plots.
    - PCA analysis for dimensionality reduction.
    - Domain-specific range checks for EMG, acceleration, and rotational columns.
    
    Parameters:
    - df: pandas DataFrame to analyze.
    - dataset_name: a string name for the dataset (used in print statements).
    - display_plots: if True, displays matplotlib plots (set to False for non-interactive environments).
    """
    print(f"\n\n----- Detailed EDA for {dataset_name} -----")
    
    # ---------------------------
    # Basic Data Overview
    # ---------------------------
    print("Shape:", df.shape)
    
    # Duplicate row check
    dup_count = df.duplicated().sum()
    print(f"\nDuplicate Rows: {dup_count} duplicate rows found.")
    
    print("\nDataFrame Info:")
    df.info()
    
    print("\nDescriptive Statistics:")
    print(df.describe(include='all').T)
    
    print("\nMissing Values:")
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if missing.empty:
        print("No missing values found.")
    else:
        print(missing)
    
    # ---------------------------
    # Identify Numeric and Datetime Columns
    # ---------------------------
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    # Filter out timedelta columns if any
    numeric_cols = [col for col in numeric_cols if not pd.api.types.is_timedelta64_dtype(df[col])]
    
    datetime_cols = df.select_dtypes(include=['datetime', 'datetimetz']).columns.tolist()
    
    # ---------------------------
    # Check for Datetime Consistency
    # ---------------------------
    if datetime_cols:
        print("\nDatetime Consistency Checks:")
        for col in datetime_cols:
            # Check if the datetime column is sorted
            if not df[col].is_monotonic_increasing:
                print(f"Warning: The datetime column '{col}' is not in increasing order.")
            else:
                print(f"The datetime column '{col}' is properly ordered.")
    
    # ---------------------------
    # Normality Test (Shapiro-Wilk) for Numeric Columns
    # ---------------------------
    print("\nNormality Test (Shapiro-Wilk) for numeric columns (p-values):")
    for col in numeric_cols:
        try:
            data = df[col].dropna()
            if len(data) > 5000:
                data = data.sample(5000, random_state=42)
            stat, p_value = stats.shapiro(data)
            print(f"{col}: p-value = {p_value:.4f}")
        except Exception as e:
            print(f"{col}: Error in normality test: {e}")
    
    # ---------------------------
    # Outlier Detection using IQR Method
    # ---------------------------
    print("\nOutlier Detection using IQR method:")
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        print(f"{col}: {outlier_count} outliers detected.")
    
    # ---------------------------
    # Demonstrate Standardization and Normalization
    # ---------------------------
    print("\nStandardization (first 5 rows, Z-scores):")
    standardized = df[numeric_cols].apply(lambda x: (x - x.mean()) / x.std())
    print(standardized.head())
    
    print("\nNormalization (first 5 rows, scaled between 0 and 1):")
    normalized = df[numeric_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    print(normalized.head())
    
    # ---------------------------
    # Dispersion Metrics for Numeric Columns
    # ---------------------------
    print("\nDispersion and Disparity Metrics for numeric columns:")
    for col in numeric_cols:
        mean_val = df[col].mean()
        std_val = df[col].std()
        var_val = df[col].var()
        min_val = df[col].min()
        max_val = df[col].max()
        range_val = max_val - min_val
        print(f"{col}: mean = {mean_val:.4f}, std = {std_val:.4f}, variance = {var_val:.4f}, range = {range_val:.4f}")
    
    # ---------------------------
    # Correlation Matrix
    # ---------------------------
    print("\nCorrelation Matrix (numeric columns):")
    corr_matrix = df[numeric_cols].corr()
    print(corr_matrix)
    
    # ---------------------------
    # Visual Exploratory Analysis
    # ---------------------------
    print("\nGenerating visual exploratory plots (if display_plots=True)...")
    if display_plots:
        for col in numeric_cols:
            plt.figure(figsize=(12, 4))
            
            # Histogram and density plot
            plt.subplot(1, 3, 1)
            sns.histplot(df[col].dropna(), kde=True)
            plt.title(f'Histogram & KDE: {col}')
            
            # Q-Q plot
            plt.subplot(1, 3, 2)
            stats.probplot(df[col].dropna(), dist="norm", plot=plt)
            plt.title(f'Q-Q Plot: {col}')
            
            # Boxplot
            plt.subplot(1, 3, 3)
            sns.boxplot(x=df[col].dropna())
            plt.title(f'Boxplot: {col}')
            
            plt.tight_layout()
            plt.show()
    
    # ---------------------------
    # Multicollinearity Check using VIF
    # ---------------------------
    if len(numeric_cols) > 1:
        print("\nMulticollinearity Check using VIF:")
        # Prepare data for VIF calculation (drop missing values)
        df_numeric = df[numeric_cols].dropna()
        vif_data = pd.DataFrame()
        vif_data["Feature"] = df_numeric.columns
        vif_data["VIF"] = [variance_inflation_factor(df_numeric.values, i) 
                           for i in range(len(df_numeric.columns))]
        print(vif_data)
    
    # ---------------------------
    # Time Series Specific Checks (if applicable)
    # ---------------------------
    if datetime_cols:
        print("\nTime Series Analysis:")
        # For each datetime column, try to find an associated numeric column for time series checks.
        for dt_col in datetime_cols:
            # Find numeric columns that might be associated (heuristic: columns that are not IDs)
            candidate_cols = [col for col in numeric_cols if 'id' not in col.lower()]
            for num_col in candidate_cols:
                # Sort by datetime column
                ts_data = df[[dt_col, num_col]].dropna().sort_values(by=dt_col)
                if ts_data.shape[0] > 30:  # ensure enough data points
                    # Stationarity test: ADF test
                    try:
                        adf_result = adfuller(ts_data[num_col])
                        print(f"ADF Test for {num_col} (sorted by {dt_col}): p-value = {adf_result[1]:.4f}")
                    except Exception as e:
                        print(f"Error performing ADF test on {num_col}: {e}")
                    
                    # If display_plots is True, plot ACF and PACF
                    if display_plots:
                        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
                        plot_acf(ts_data[num_col], ax=axes[0], lags=20)
                        axes[0].set_title(f'ACF of {num_col}')
                        plot_pacf(ts_data[num_col], ax=axes[1], lags=20)
                        axes[1].set_title(f'PACF of {num_col}')
                        plt.tight_layout()
                        plt.show()
                    # Only process one candidate per datetime column
                    break

    # ---------------------------
    # PCA Analysis for Numeric Data
    # ---------------------------
    if len(numeric_cols) > 1:
        print("\nPCA Analysis on numeric columns:")
        # Drop NA values for PCA and standardize data
        pca_data = df[numeric_cols].dropna()
        standardized_data = (pca_data - pca_data.mean()) / pca_data.std()
        try:
            pca = PCA()
            pca.fit(standardized_data)
            explained_variance = pca.explained_variance_ratio_
            for idx, var in enumerate(explained_variance):
                print(f"PC{idx+1}: {var*100:.2f}% of variance explained")
        except Exception as e:
            print("Error performing PCA:", e)
    
    # ---------------------------
    # Domain-Specific Range Checks
    # ---------------------------
    print("\nDomain-Specific Range Checks:")
    domain_keywords = ['emg', 'accel', 'rotat']
    for col in numeric_cols:
        if any(keyword in col.lower() for keyword in domain_keywords):
            col_min = df[col].min()
            col_max = df[col].max()
            print(f"{col} (domain-specific): min = {col_min}, max = {col_max}")
            # Here you can add further domain-specific validations if needed.
    
    print("\n----- End of EDA for", dataset_name, "-----\n")


# ------------------ New EDA Calls ------------------
# Run the detailed EDA on each dataset
detailed_eda(emg_pitch_data, dataset_name="EMG Pitch Data")
detailed_eda(summarized_bio, dataset_name="Summarized Biomechanical Profile")
detailed_eda(granular_data, dataset_name="Granular Dataset")

# Optionally, you can clean missing values and then re-run the EDA:
# cleaned_emg = handle_missing_values(emg_pitch_data)
# detailed_eda(cleaned_emg, dataset_name="EMG Pitch Data (Cleaned)")




----- Detailed EDA for EMG Pitch Data -----
Shape: (134720, 38)

Duplicate Rows: 0 duplicate rows found.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134720 entries, 0 to 134719
Data columns (total 38 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   EMG 1 (mV) - FDS (81770)                 134720 non-null  float64       
 1   ACC X (G) - FDS (81770)                  134720 non-null  float64       
 2   ACC Y (G) - FDS (81770)                  134720 non-null  float64       
 3   ACC Z (G) - FDS (81770)                  134720 non-null  float64       
 4   GYRO X (deg/s) - FDS (81770)             134720 non-null  float64       
 5   GYRO Y (deg/s) - FDS (81770)             134720 non-null  float64       
 6   GYRO Z (deg/s) - FDS (81770)             134720 non-null  float64       
 7   EMG 1 (mV) - FCU (81728)                 134720 non-null  

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


EMG 1 (mV) - FCU (81728): 24133 outliers detected.
ACC X (G) - FCU (81728): 4750 outliers detected.
ACC Y (G) - FCU (81728): 1009 outliers detected.
ACC Z (G) - FCU (81728): 8316 outliers detected.
GYRO X (deg/s) - FCU (81728): 13416 outliers detected.
GYRO Y (deg/s) - FCU (81728): 9811 outliers detected.
GYRO Z (deg/s) - FCU (81728): 14406 outliers detected.
EMG 1 (mV) - FCR (81745): 9536 outliers detected.
ACC X (G) - FDS (81770)_spike_flag: 0 outliers detected.
ACC X (G) - FCU (81728)_spike_flag: 0 outliers detected.
ACC Y (G) - FDS (81770)_spike_flag: 0 outliers detected.
ACC Y (G) - FCU (81728)_spike_flag: 30414 outliers detected.
ACC Z (G) - FDS (81770)_spike_flag: 0 outliers detected.
ACC Z (G) - FCU (81728)_spike_flag: 0 outliers detected.
GYRO X (deg/s) - FDS (81770)_spike_flag: 2378 outliers detected.
GYRO X (deg/s) - FCU (81728)_spike_flag: 2185 outliers detected.
GYRO Y (deg/s) - FDS (81770)_spike_flag: 1867 outliers detected.
GYRO Y (deg/s) - FCU (81728)_spike_flag: 2000 o

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\regression\linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


                                    Feature        VIF
0                  EMG 1 (mV) - FDS (81770)   1.455843
1                   ACC X (G) - FDS (81770)   8.587782
2                   ACC Y (G) - FDS (81770)  13.345326
3                   ACC Z (G) - FDS (81770)   7.955110
4              GYRO X (deg/s) - FDS (81770)  10.368718
5              GYRO Y (deg/s) - FDS (81770)  15.331578
6              GYRO Z (deg/s) - FDS (81770)  11.861217
7                  EMG 1 (mV) - FCU (81728)   1.027031
8                   ACC X (G) - FCU (81728)   8.906968
9                   ACC Y (G) - FCU (81728)  12.667034
10                  ACC Z (G) - FCU (81728)  12.947673
11             GYRO X (deg/s) - FCU (81728)   9.897152
12             GYRO Y (deg/s) - FCU (81728)  15.211898
13             GYRO Z (deg/s) - FCU (81728)  13.598031
14                 EMG 1 (mV) - FCR (81745)   1.024942
15       ACC X (G) - FDS (81770)_spike_flag   5.765958
16       ACC X (G) - FCU (81728)_spike_flag   4.582558
17       A

...

Error performing PCA: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Domain-Specific Range Checks:
avg_shoulder_rotation (domain-specific): min = -1.82581162, max = 101.36335313
max_shoulder_rotation_speed (domain-specific): min = 80.1447, max = 7593.1172
max_torso_rotation_speed (domain-specific): min = 69.0322, max = 948.4239
shoulder_rotation_variability (domain-specific): min = 15.50099202, max = 42.49240572
max_

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


GYRO Z (deg/s) - FDS (81770)_spike_flag: p-value = 0.0000
GYRO Z (deg/s) - FCU (81728)_spike_flag: p-value = 0.0000
EMG 1 (mV) - FDS (81770)_spike_flag: p-value = 0.0000
EMG_high_flag: p-value = 0.0000
EMG_low_flag: p-value = 0.0000
EMG_extreme_flag: p-value = 0.0000
EMG_extreme_flag_dynamic: p-value = 1.0000
ThrowingMotion: p-value = 0.0000
session_biomech: p-value = 1.0000
trial_biomech: p-value = 0.0000
ongoing_timestamp_biomech: p-value = 1.0000
shoulder_angle_x_biomech: p-value = 0.0000
shoulder_angle_y_biomech: p-value = 0.0000
shoulder_angle_z_biomech: p-value = 0.0000
elbow_angle_x_biomech: p-value = 0.0000
elbow_angle_y_biomech: p-value = 0.0000
elbow_angle_z_biomech: p-value = 0.0000
torso_angle_x_biomech: p-value = 0.0000
torso_angle_y_biomech: p-value = 0.0000
torso_angle_z_biomech: p-value = 0.0000
pelvis_angle_x_biomech: p-value = 0.0000
pelvis_angle_y_biomech: p-value = 0.0000
pelvis_angle_z_biomech: p-value = 0.0000
shoulder_velo_x_biomech: p-value = 0.0000
shoulder_vel

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                                          Feature  VIF
0                        EMG 1 (mV) - FDS (81770)  1.0
1                         ACC X (G) - FDS (81770)  1.0
2                         ACC Y (G) - FDS (81770)  1.0
3                         ACC Z (G) - FDS (81770)  1.0
4                    GYRO X (deg/s) - FDS (81770)  1.0
..                                            ...  ...
67                         elbow_moment_z_biomech  1.0
68               shoulder_thorax_moment_x_biomech  1.0
69               shoulder_thorax_moment_y_biomech  1.0
70               shoulder_thorax_moment_z_biomech  1.0
71  max_shoulder_internal_rotational_velo_biomech  1.0

[72 rows x 2 columns]

Time Series Analysis:
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by Date/Time_parsed): p-value = 0.0000
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by Timestamp_parsed): p-value = 0.0000
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by datetime): p-value = 0.0000
ADF Test for EMG 1 (mV) - FDS (81770) (sorted b

# Feature Engineering

metrics to add:
* Set up phases based on the pitcher movement so we can set them into different phases
* add the phases to the EMG dataset so we can better predict on that and create use the merge_asof dataset to see if it's worth predicting with for the time series motion of the joints (the merge_asof would need to be a small difference)
* add in pulse data if we don't have arm slot/torque/speed in the database
* add in vulgas range and possibly the ongoing detriment from that
* finish up the sql statement (ensure it's completely validated) for the base table
* separate the dataset into time series classification, regression and tree based classification datasets
* check the feaures importances and such
* use permutation importance and use github(dot)com/eli5-org/eli5

Setting up feature dictionary to speak over each feature to talk about actual important metrics outside of those feature selected
Feature Selection (RFE, SHAP importance, correlation (to take out multi-collinearity metrics or combine them))
